In [487]:
import json
import pandas as pd
import numpy as np
import datetime as dt

In [488]:
query='christmas'

In [489]:
output=dict()

In [490]:
output['word']=query

In [491]:
# Open JSON <---- THIS WILL BE REPLACED WITH MONGO QUERY
with open('d3demo/json/'+word+'.json', "r") as read_file:
    data = json.load(read_file)

In [492]:
# Set columns for df
columns=['counts','rank','time']

In [493]:
# Build df
df=pd.DataFrame(data, columns=columns)

In [494]:
# Calculate min, max, and mean rank
output['maxrank']=min(df['rank'].values)
output['minrank']=max(df['rank'].values)
output['meanrank']=np.round(np.mean(df['rank'].values)).astype(int)

In [495]:
# Pull out beginning part of date (YY-MM-DD)
df['time'] = [t[:10] for t in df['time']]
# Convert date to date object
df['time'] = [dt.datetime.strptime(t, '%Y-%m-%d').date() for t in df['time']]
# Index df by date
df.set_index('time',inplace=True)
# Find today's date
dt.datetime.today().date()
# How many days should we have in our dataset?
daterange=(dt.datetime.today().date()-df.index.min()).days
# Which days are we missing data for?
# Get a list of all days between today and the beginning of time...
missingdays=[(dt.datetime.today() - dt.timedelta(days=x+1)).date() for x in range(daterange)]
# ... then remove any days that we *do* have in our data from our list of missing dates:
for day in df.index:
    missingdays.remove(day)
# Fill any missing days with placeholder values
for day in missingdays:
    df.loc[day] = None
    df.loc[day]['counts'] = 0
    df.loc[day]['rank'] = 999999999
# Sort days
df.sort_values(by='time',ascending=False,inplace=True)

In [496]:
# Convert time back to a string
df.index=[t.strftime("%Y-%m-%d") for t in df.index]

In [497]:
# Send dates, ranks and counts as arrays to the output dict
output['times']=df.index.values
output['ranks']=df['rank'].values
output['counts']=df['counts'].values

In [498]:
output

{'word': 'christmas',
 'maxrank': 15,
 'minrank': 41858,
 'meanrank': 13129,
 'times': array(['2019-07-31', '2019-07-30', '2019-07-29', ..., '2006-11-02',
        '2006-11-01', '2006-10-31'], dtype=object),
 'ranks': array([9.99999999e+08, 9.99999999e+08, 9.99999999e+08, ...,
        9.99999999e+08, 2.60000000e+01, 1.50000000e+01]),
 'counts': array([ 0.,  0.,  0., ...,  0., 20., 34.])}